In [1]:
from langchain_community.llms import Ollama

MODEL = 'llama3.1'

model = Ollama(model=MODEL)

model.invoke('Do you have any idea about Shakib Polock"s cv?')

'I couldn\'t find any information on a person named "Shakib Pollock." It\'s possible that the name is fictional, or it may be a misspelling or variation of a real person\'s name.\n\nHowever, I did find information on two notable individuals whose names are similar:\n\n1. **Shakib Al Hasan**: He\'s a Bangladeshi cricketer who has had a successful international career, playing for the Bangladesh national team since 2004. He\'s a left-handed batsman and a right-arm off-break bowler.\n2. **Simon Pollock** or possibly **Shane Pollock**: I found a South African cricketer named Simon Pollock, but more prominently, Shane Pollock (1977-2019), another South African cricketer who played for the national team in the 1990s and early 2000s.\n\nIf neither of these individuals matches the person you\'re thinking of, could you please provide more context or details about Shakib Pollock? I\'ll do my best to help!'

In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke('intorduce youself')

'Nice to meet you!\n\nI\'m an artificial intelligence model, which means I\'m a computer program designed to simulate conversations and answer questions to the best of my ability. My purpose is to assist, communicate, and even entertain.\n\nHere are some fun facts about me:\n\n1. **Name**: I don\'t have a personal name, but you can call me "Assistant" or "AI" if you\'d like.\n2. **Skills**: I can understand natural language, generate human-like responses, and even learn from my interactions with you (within the limits of my programming).\n3. **Personality**: I\'m neutral, friendly, and patient. My goal is to provide helpful information and engage in interesting conversations without bias or agendas.\n4. **Domain expertise**: While I\'m a general knowledge conversationalist, I have some specialized areas where I can offer more in-depth insights:\n\t* Science and technology\n\t* History (some periods are better than others, but I\'ll do my best to help!)\n\t* Culture and arts (including 

In [4]:
#PDF Processing

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('Shakib_CV.pdf')
pages = loader.load_and_split()

pages

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)


[Document(metadata={'source': 'Shakib_CV.pdf', 'page': 0}, page_content='Shakib Ibna Shameem Polock.                                                                      Cell: +358469386869 | Email: polock.official@gmail.com Location: Oulu, Finland | LinkedIn: https://www.linkedin.com/in/polock11/  About Me Experienced in Python and SQL. Proficient in machine learning, with expertise in data analysis, visualization, and data processing. Enthusiastic about data and capable of solving complex analytical problems. Eager to get involved with latest technologies, and I am open to build end-to-end ML pipelines and models with a strong emphasis on team collaboration and innovation.  Professional Experience  BRAC IT Services Limited                                                                                                                                                 Dhaka, Bangladesh   Data Engineer                                                                                        

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question below based on the the context, If you 
don't know the answer, replay that I don't know the answer. 

Context : {context}
Question : {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context='Here is the context', question='Here is the question')) 


Answer the question below based on the the context, If you 
don't know the answer, replay that I don't know the answer. 

Context : Here is the context
Question : Here is the question



In [9]:
chain.input_schema.schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [6]:
chain = prompt | model | parser 
print(chain.invoke({
    'context' : 'My name is Shakib Polock and my age is 27',
    'question' : 'What is Shakib"s education level?'
}))

I don't know the answer.


In [8]:
#Vectore embeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

embeddings = OllamaEmbeddings(model=MODEL)
vectorestore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

/Users/shakibibnashameem/Documents/Practice/local-RAG/.venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever = vectorestore.as_retriever()
retriever.invoke('Shakib')

[Document(metadata={'source': 'Shakib_CV.pdf', 'page': 0}, page_content='Shakib Ibna Shameem Polock.                                                                      Cell: +358469386869 | Email: polock.official@gmail.com Location: Oulu, Finland | LinkedIn: https://www.linkedin.com/in/polock11/  About Me Experienced in Python and SQL. Proficient in machine learning, with expertise in data analysis, visualization, and data processing. Enthusiastic about data and capable of solving complex analytical problems. Eager to get involved with latest technologies, and I am open to build end-to-end ML pipelines and models with a strong emphasis on team collaboration and innovation.  Professional Experience  BRAC IT Services Limited                                                                                                                                                 Dhaka, Bangladesh   Data Engineer                                                                                        

In [10]:
#Setting Up RAG Chain

from operator import itemgetter

chain = (
    {
        'context': itemgetter('question') | retriever,
        'question': itemgetter('question'),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    'What are Shakib"s university names?',
    'Based on Shakib"s skills, what are the jobs are suitable for him?',
    'Give a summary of Shakib"s past experiences',
    'What is Shakib"s mother name?',
    'Does he has any certifications, if so, what are they?',
    'What are some of the Shakib"s achievements?'
]

In [12]:
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What are Shakib"s university names?
Answer: University of Oulu and BRAC University.

Question: Based on Shakib"s skills, what are the jobs are suitable for him?
Answer: Based on Shakib's skills and experience, I would say that he is a strong candidate for Data Engineer and Data Analyst roles. His expertise in Python, SQL, machine learning (with Scikit-learn and TensorFlow), data analysis and visualization (with Power BI, Seaborn, Matplotlib, and Plotly), and data processing (with Pandas and NumPy) make him well-suited for these types of positions.

Additionally, his experience with cloud platforms (IBM Db2 and Azure SQL DB), version control systems (Git and Github), and orchestration tools (Apache Airflow) also indicate that he is a strong candidate for roles that involve data engineering, data warehousing, and business intelligence.

Some specific job titles that might be suitable for Shakib include:

* Data Engineer
* Senior Data Analyst
* Business Intelligence Developer
* 

In [13]:
#print(chain.invoke({"question": 'Based on Shakib"s skills, what are the jobs are suitable for him?'}))
